In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical
from keras import Model

from constants import img_size
from utils.models import get_base_model

In [ ]:
def generator(datagen, data_dir, batch_size, **kwargs):
    print('Data directory:', data_dir)
    return datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=0,
        **kwargs,
    )

In [ ]:
def paths(name):
    paths_to_create = [
        '../input/features/' + name,
    ]
    
    for path in paths_to_create:
        if not os.path.exists(path):
            os.makedirs(path)
            
    return paths_to_create[0]

In [ ]:
def extractor(model_constructor, name, preprocessor, batch_size, source_dir_train, source_dir_dev, pooling=None, layer=None, verbose = True):
    model = get_base_model(model_constructor, img_size, pooling, verbose, layer)

    datagen = ImageDataGenerator(preprocessing_function=preprocessor)
    generator_train = generator(datagen, source_dir_train, batch_size)
    generator_dev = generator(datagen, source_dir_dev, batch_size)
    
    path = paths(name)
    print('Path:', path)
    np.save(path + '/filenames_train.npy', generator_train.filenames)
    np.save(path + '/classes_train.npy', generator_train.classes)
    np.save(path + '/filenames_dev.npy', generator_dev.filenames)
    np.save(path + '/classes_dev.npy', generator_dev.classes)
    
    #train features
    features_train = model.predict_generator(generator_train, len(generator_train), verbose=1)
    np.save(path + '/features_train.npy', features_train)
    
    #dev features
    features_dev = model.predict_generator(generator_dev, len(generator_dev), verbose=1)
    np.save(path + '/features_dev.npy', features_dev)

## Extracting

In [ ]:
extractor(
    model_constructor=ResNet50, 
    name='res_net', 
    preprocessor=resnet50_preprocess_input, 
    batch_size=8, 
#    layer=16,
    pooling='avg',
    source_dir_train='../input/datasets/train_dataset/',
    source_dir_dev='../input/datasets/dev_dataset/',
)

In [ ]:
extractor(
    model_constructor=InceptionV3, 
    name='inception', 
    preprocessor=inception_preprocess_input, 
    batch_size=8, 
#    layer=16,
    pooling='avg',
    source_dir_train='../input/datasets/train_dataset/',
    source_dir_dev='../input/datasets/dev_dataset/',
)

In [ ]:
extractor(
    model_constructor=Xception, 
    name='xception', 
    preprocessor=xception_preprocess_input, 
    batch_size=8, 
#    layer=16,
    pooling='avg',
    source_dir_train='../input/datasets/train_dataset/',
    source_dir_dev='../input/datasets/dev_dataset/',
)

## Saving models

In [ ]:
model = get_base_model(Xception, img_size, pooling='avg', verbose=1)

In [ ]:
model.save('../input/base_models/xception.h5')